<a href="https://colab.research.google.com/github/h40300965/deep-learnin/blob/main/Copy_of_Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sam-tf --quiet
!pip install tensorflow-addons --quiet

ERROR: Could not find a version that satisfies the requirement sam-tf (from versions: none)
ERROR: No matching distribution found for sam-tf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow import keras

class SAMModel(keras.Model):
    def __init__(self, base_model, rho=0.05):
        super().__init__()
        self.base_model = base_model
        self.rho = rho  # Perturbation radius (called "distance" in your code)

    def train_step(self, data):
        (images, labels) = data

        # First forward-backward pass
        with tf.GradientTape() as tape:
            logits = self.base_model(images, training=True)
            loss = self.compiled_loss(labels, logits)
        gradients = tape.gradient(loss, self.base_model.trainable_variables)

        # Apply SAM perturbation
        grad_norm = tf.linalg.global_norm(gradients)
        scale = self.rho / (grad_norm + 1e-12)
        perturbations = [g * scale for g in gradients]

        # Save original weights and apply perturbations
        original_weights = [tf.identity(w) for w in self.base_model.weights]
        for w, p in zip(self.base_model.trainable_variables, perturbations):
            w.assign_add(p)

        # Second forward-backward pass
        with tf.GradientTape() as tape:
            logits = self.base_model(images, training=True)
            loss = self.compiled_loss(labels, logits)
        gradients = tape.gradient(loss, self.base_model.trainable_variables)

        # Restore original weights
        for w, orig in zip(self.base_model.weights, original_weights):
            w.assign(orig)

        # Update weights with the gradients from the second pass
        self.optimizer.apply_gradients(
            zip(gradients, self.base_model.trainable_variables)
        )

        # Update metrics
        self.compiled_metrics.update_state(labels, logits)
        return {m.name: m.result() for m in self.metrics}

In [ ]:
from tensorflow import keras

# Replace this with your actual NFNet builder
def build_nfnet(input_shape=(224, 224, 3), num_classes=10):
    return keras.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(num_classes)
    ])

# Build model and wrap with SAM
base_model = build_nfnet(num_classes=10)
sam_model = SAMModel(base_model, rho=0.05)  # Use rho=0.05 instead of distance

# Compile and train
sam_model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=3e-4, weight_decay=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models, mixed_precision
#from sam_tf import SAMModel
import numpy as np


In [ ]:
# Enable Mixed Precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


In [ ]:
# Scaled Weight Standardization Conv2D Layer
class ScaledWSConv2D(layers.Conv2D):
    def build(self, input_shape):
        super().build(input_shape)
        kernel_shape = self.kernel.shape.as_list()
        self.fan_in = np.prod(kernel_shape[:-1])
        self.scale = tf.math.sqrt(2. / self.fan_in)

    def call(self, inputs):
        mean, var = tf.nn.moments(self.kernel, axes=[0, 1, 2], keepdims=True)
        weight = (self.kernel - mean) * tf.math.rsqrt(var + 1e-10)
        x = tf.nn.conv2d(inputs, filters=weight * self.scale,
                         strides=self.strides, padding=self.padding.upper())
        if self.use_bias:
            x = tf.nn.bias_add(x, self.bias)
        return x

In [ ]:
#with SkipInit
class NFBlock(layers.Layer):
    def __init__(self, channels, expansion=2, stride=1, **kwargs):
        super().__init__(**kwargs)
        mid_channels = channels // expansion
        self.conv1 = ScaledWSConv2D(mid_channels, 1, strides=stride, use_bias=False)
        self.act1 = layers.Activation('gelu')

        self.conv2 = ScaledWSConv2D(mid_channels, 3, strides=1, padding='same', use_bias=False)
        self.act2 = layers.Activation('gelu')

        self.conv3 = ScaledWSConv2D(channels, 1, strides=1, use_bias=False)
        self.act3 = layers.Activation('gelu')

        if stride != 1 or inputs.shape[-1] != channels:
            self.shortcut = ScaledWSConv2D(channels, 1, strides=stride, use_bias=False)
        else:
            self.shortcut = tf.identity

        self.skip_gain = tf.Variable(0., trainable=True, name="skip_gain")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.act1(x)

        x = self.conv2(x)
        x = self.act2(x)

        x = self.conv3(x)
        x = x * self.skip_gain

        shortcut = self.shortcut(inputs)
        x = x + shortcut
        x = self.act3(x)
        return x

In [ ]:
# Stem Network
def get_stem(filters):
    return models.Sequential([
        layers.Input((224, 224, 3)),
        ScaledWSConv2D(filters, 3, strides=2, padding='same'),
        layers.Activation('gelu'),
        ScaledWSConv2D(filters * 2, 3, strides=1, padding='same'),
        layers.Activation('gelu'),
        ScaledWSConv2D(filters * 4, 3, strides=1, padding='same'),
        layers.Activation('gelu'),
        layers.MaxPool2D(3, strides=2, padding='same')
    ], name="stem")



In [ ]:
# Build Full NFNet Model
def build_nfnet(num_classes=1000):
    stem = get_stem(64)
    blocks = [
        NFBlock(256),
        NFBlock(256),
        NFBlock(256),
        NFBlock(512, stride=2),
        NFBlock(512),
        NFBlock(512),
        NFBlock(1024, stride=2),
        NFBlock(1024),
        NFBlock(1024)
    ]

    head = models.Sequential([
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes)
    ])

    model = models.Sequential([
        stem,
        *blocks,
        head
    ])
    return model



In [ ]:
# Create model
def build_nfnet(input_shape=(224, 224, 3), num_classes=10):
    return keras.Sequential([
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(num_classes)
    ])

# Build model and wrap with SAM
#base_model = build_nfnet(num_classes=10)
#sam_model = SAMModel(base_model, rho=0.05)  # Use rho=0.05 instead of dist
model= build_nfnet(num_classes=1000)
sam_model = SAMModel(model, rho=0.05)


In [ ]:
# Compile
sam_model.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=3e-4, weight_decay=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


In [ ]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)                    │ (None, 222, 222, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_9           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1000)                │          65,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 66,792 (260.91 KB)

 Trainable params: 66,792 (260.91 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
sam_model.summary()

Model: "sam_model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_9 (Sequential)            │ (None, 1000)                │          66,792 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 66,792 (260.91 KB)

 Trainable params: 66,792 (260.91 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
!pip install tensorflow-datasets --quiet

code imagnet tds   "imagenet21k"

In [ ]:
import tensorflow_datasets as tfds

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.image.per_image_standardization(image)
    return image, label

# Load ImageNet dataset (example: subset 'train[:1%]')
dataset_name = 'wiki_auto'
batch_size = 512
buffer_size = 1024

train_dataset = tfds.load(dataset_name, split='dev', shuffle_files=True)
train_dataset = train_dataset.map(preprocess).shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = tfds.load(dataset_name, split='validation').map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__preprocess() missing 1 required positional argument: 'label'


train model

In [ ]:
history = sam_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

NameError: name 'val_dataset' is not defined